In [ ]:
%load_ext autoreload
%autoreload 2
RootDIR = "/home/jw3514/Work/ASD_Circuits_CellType/"
import sys
sys.path.insert(1, RootDIR + 'src')
from ASD_Circuits import *
from venn import venn
import requests
import json

# DownLoad Allen IDs

Download expression data and discontinued Entrez IDs

http://api.brain-map.org/class_detail/Gene

http://api.brain-map.org/examples/doc/structures/download_data.py.html

http://help.brain-map.org/display/api/Example+Queries+for+Experiment+Metadata

http://api.brain-map.org/doc/DataSet.html

https://ftp.ncbi.nih.gov/gene/README

In [ ]:
url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=model::SectionDataSet," +  \
   "rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse']," + \
   "treatments[name$eq'ISH'],genes,plane_of_section," + \
   "rma::options,[tabular$eq'plane_of_sections.name+as+plane'," + \
   "'genes.acronym+as+gene_acronym'," + \
   "'genes.entrez_id+as+genes_entrez_id'," + \
   "'genes.ensembl_id+as+gene_ensembl_id'," + \
   "'genes.alias_tags+as+gene_alias_tags'," + \
   "'data_sets.id+as+section_data_set_id']," + \
   "[order$eq'plane_of_sections.name,genes.acronym,data_sets.id']&" + \
   "start_row=0&num_rows=all"
print(url)
r = requests.get(url, allow_redirects=True)
open(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_ISH_experiments.csv', 'wb').write(r.content)


In [ ]:
url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=model::Gene," +  \
   "rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse']," + \
   "[tabular$eq'acronym+as+gene_acronym'," + \
   "'gene_aliases.name+as+gene_aliases'," + \
   "'entrez_id+as+genes_entrez_id'," + \
   "[order$eq'acronym']&" + \
   "start_row=0&num_rows=all"
print(url)
r = requests.get(url, allow_redirects=True)
open(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_ISH_genes.csv', 'wb').write(r.content)


In [ ]:
url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=" + \
        "model::Gene," + \
        "rma::criteria,products[abbreviation$eq'Mouse']," + \
        "rma::options,[tabular$eq'genes.id','genes.acronym+as+gene_symbol'," + \
        "'genes.name+as+gene_name','genes.entrez_id+as+entrez_gene_id'," + \
        "'genes.homologene_id+as+homologene_group_id'," + \
        "'genes.alias_tags+as+gene_alias_tags']," + \
        "[order$eq'genes.acronym']&num_rows=all&start_row=0"
print(url)
r = requests.get(url, allow_redirects=True)
open(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_Genes.csv', 'wb').write(r.content)


Download data from query results (expression energy for each section ID) can be downloaded using:

`script/download_ISH.py

This script retrieves the expression energy values for each section ID from the Allen Brain Atlas API.

Downloaded ISH expression energy data should be saved at dat/allen-mouse-exp/allen-mouse-brain-altas-str-unionzed

# Mouse To Human Mapping

In [ ]:


file_path = RootDIR + 'dat/HOM_MouseHumanSequence.rpt'
if not os.path.exists(file_path):
    url = "https://www.informatics.jax.org/downloads/reports/HOM_MouseHumanSequence.rpt"
    r = requests.get(url, allow_redirects=True)
    open(file_path, 'wb').write(r.content)

In [ ]:
AllenMouseGenes = pd.read_csv(RootDIR + 'dat/allen-mouse-exp/All_Mouse_Brain_ISH_experiments.csv')
Human2MouseHom = pd.read_csv(RootDIR + 'dat/HOM_MouseHumanSequence.rpt', delimiter="\t")

In [ ]:
Homo_IDs = set(Human2MouseHom["DB Class Key"].values)
print(len(Homo_IDs))

In [ ]:
Human2Mouse_Genes = {}
Mouse2Human_Genes = {}
# Key:EntrezID
# symbol: FOXP1
# mouseHomo: [(), ()]
# allen_section_data_set_id: [1,2]
for counter, ID in enumerate(Homo_IDs):
    tmp_df = Human2MouseHom[Human2MouseHom["DB Class Key"]==ID]
    #print(tmp_df)   
    hum_genes, mou_genes = [], []
    for i, row in tmp_df.iterrows():
        TaxonID = row["NCBI Taxon ID"]
        Symbol = row["Symbol"]
        Entrez = row["EntrezGene ID"]
        if  TaxonID == 9606:
            hum_genes.append((Symbol, Entrez))
        elif TaxonID == 10090:
            mou_genes.append((Symbol, Entrez))
    # Fill up Human2Mouse_Genes
    for (Symbol, Entrez) in hum_genes:
        if Entrez not in Human2Mouse_Genes:
            Human2Mouse_Genes[Entrez] = {}#mou_genes
            Human2Mouse_Genes[Entrez]["symbol"] = Symbol
            Human2Mouse_Genes[Entrez]["mouseHomo"] = mou_genes
        else:
            Human2Mouse_Genes[Entrez]["mouseHomo"].extend(mou_genes)
        
    # Fill up Mouse2Human_Genes
    for (Symbol, Entrez) in mou_genes:
        if Entrez not in Mouse2Human_Genes:
            Mouse2Human_Genes[Entrez] = {} #mou_genes
            Mouse2Human_Genes[Entrez]["symbol"] = Symbol
            Mouse2Human_Genes[Entrez]["humanHomo"] = hum_genes
            Mouse2Human_Genes[Entrez]["allen_section_data_set_id"] = []
        else:
            Mouse2Human_Genes[Entrez]["humanHomo"].extend(hum_genes)
            

In [ ]:
Mouse2Human_Genes_2 = {}
for k,v in Mouse2Human_Genes.items():
    Mouse2Human_Genes_2[v["symbol"]] = {}
    Mouse2Human_Genes_2[v["symbol"]]["Entrez"] = k
    Mouse2Human_Genes_2[v["symbol"]]["humanHomo"] = v["humanHomo"]

In [ ]:
pk.dump(Mouse2Human_Genes_2, open(RootDIR + 'dat/Mouse2Human_Symbol.pk', "wb"))
pk.dump(Mouse2Human_Genes, open(RootDIR + 'dat/Mouse2Human_Entrez.pk', "wb"))


In [ ]:
Discontinued_ID = {}
df = pd.read_csv(RootDIR + 'dat/gene_history.human.mouse.tsv', delimiter="\t")
Discontinued_ID = dict(zip(df["Discontinued_GeneID"].values, df["GeneID"].values))


In [ ]:
# Match Allen section_id to mouse genes with Entrez ID
def MatchAllen(symbol, entrez, row):
    # row: plane,gene_acronym,genes_entrez_id,gene_ensembl_id,gene_alias_tags,section_data_set_id
    #print(symbol, entrez)
    #print(row["gene_acronym"], row["genes_entrez_id"], row["gene_alias_tags"] )
    if str(row["genes_entrez_id"]) != "nan" and int(entrez) == int(row["genes_entrez_id"]):
        return row["section_data_set_id"]
    elif symbol == row["gene_acronym"]:
        return row["section_data_set_id"]
    else:
        if str(row["gene_alias_tags"]) == "nan":
            #print("haha")
            return None
        #print(str(row["gene_alias_tags"]), type(row["gene_alias_tags"]))
        for alias in row["gene_alias_tags"].split():
            alias = alias.strip()
            if alias == symbol:
                return row["section_data_set_id"]
    return None

Entrez_Failed_ID = []
for i, row in AllenMouseGenes.iterrows():
    if row["section_data_set_id"] == 68269058:
        print(row)
    if str(row["gene_acronym"]) != "nan":
        allen_symbol = row["gene_acronym"]
    else:
        allen_symbol = None
    if str(row["genes_entrez_id"]) != "nan":
        allen_entrez = int(row["genes_entrez_id"])
    else:
        allen_entrez = None
    if str(row["gene_alias_tags"]) != "nan":
        allen_alias = row["gene_alias_tags"].split()
    else:
        allen_alias = []
    allen_section_id = row["section_data_set_id"]
    try:
        potential_ID = int(Discontinued_ID[allen_entrez])
    except:
        potential_ID = -1
        pass
    if allen_entrez in Mouse2Human_Genes:
        Mouse2Human_Genes[allen_entrez]["allen_section_data_set_id"].append(allen_section_id)
    elif potential_ID in Mouse2Human_Genes:
        Mouse2Human_Genes[potential_ID]["allen_section_data_set_id"].append(allen_section_id)
    else:    
        Entrez_Failed_ID.append(allen_section_id)
 

In [ ]:
Mouse2Human_Genes[18430]

In [ ]:
68269058 in Entrez_Failed_ID

In [ ]:
Mouse2Human_Genes

In [ ]:
print(len(Entrez_Failed_ID))

In [ ]:
# Check missed by entrez ID
Mouse_MissSectionID = []
for K, V in Mouse2Human_Genes.items():
    if len(V["allen_section_data_set_id"]) == 0:
        Mouse_MissSectionID.append(K)
print(len(Mouse_MissSectionID))

In [ ]:
Unlinked_AllenMouseGenes = AllenMouseGenes[AllenMouseGenes["section_data_set_id"].isin(
    Entrez_Failed_ID)]
Symbol_Failed_ID = []
Mapped_by_symbol = 0
Mapped_by_alias = 0
for i, row in Unlinked_AllenMouseGenes.iterrows():
    if row["section_data_set_id"] == 68269058:
        print(row)
    if str(row["gene_acronym"]) != "nan":
        allen_symbol = row["gene_acronym"]
    else:
        allen_symbol = None
    if str(row["gene_alias_tags"]) != "nan":
        allen_alias = row["gene_alias_tags"].split()
    else:
        allen_alias = []
    allen_section_id = row["section_data_set_id"]
    FLAG_FIND = False
    for k in Mouse_MissSectionID:
        symbol = Mouse2Human_Genes[k]["symbol"]
        if symbol.lower() == allen_symbol.lower():
            Mouse2Human_Genes[k]["allen_section_data_set_id"].append(allen_section_id)
            FLAG_FIND = True
            Mapped_by_symbol += 1
            break
        else:
            for alias in allen_alias:
                if alias == "C9":
                    print(row)
                if alias.lower() == symbol.lower():
                    Mouse2Human_Genes[k]["allen_section_data_set_id"].append(allen_section_id)
                    FLAG_FIND = True
                    Mapped_by_alias += 1
                    print(alias)
                    break
    if not FLAG_FIND:
        Symbol_Failed_ID.append(allen_section_id)
print(Mapped_by_symbol, Mapped_by_alias)
print(len(Symbol_Failed_ID))

In [ ]:
Unlinked_AllenMouseGenes = AllenMouseGenes[AllenMouseGenes["section_data_set_id"].isin(
    Symbol_Failed_ID)]

In [ ]:
Unlinked_AllenMouseGenes

In [ ]:
DIR = RootDIR + 'dat/allen-mouse-exp/'
with open(DIR + 'human2mouse.0420.json', 'w') as outfile:
    json.dump(Human2Mouse_Genes, outfile)
with open(DIR + 'mouse2sectionID.0420.json', 'w') as outfile:
    json.dump(Mouse2Human_Genes, outfile)
Unlinked_AllenMouseGenes.to_csv(DIR + "Allen_Unlinked_SectionIDs.0415.csv", index=False)

In [ ]:
Mouse_MissSectionID = []
for K, V in Mouse2Human_Genes.items():
    if len(V["allen_section_data_set_id"]) == 0:
        Mouse_MissSectionID.append(K)
print(len(Mouse_MissSectionID))

In [ ]:
STR_Meta = pd.read_csv(RootDIR + 'dat/allen-mouse-exp/allen_brain_atlas_structures.csv')
STR_Meta.dropna(inplace=True, subset=["atlas_id"])
STR_Meta["atlas_id"] = STR_Meta["atlas_id"].astype(int)
#STR_Meta["parent_structure_id"] = STR_Meta["parent_structure_id"].astype(int)
STR_Meta = STR_Meta.set_index("atlas_id")
for i, row in STR_Meta.iterrows():
    STR_Meta.loc[i, "Name2"] = modify_str(row["safe_name"])


In [ ]:
Selected_STRs = LoadList(RootDIR + 'dat/allen-mouse-exp/Structures.txt')


In [ ]:
STR_Meta_2 = STR_Meta[STR_Meta["Name2"].isin(Selected_STRs)]
STR_Meta_2 = STR_Meta_2.sort_values("Name2")
STR_Meta_2.to_csv(RootDIR + 'dat/allen-mouse-exp/Selected_213_STRs.Meta.csv')
